<a href="https://colab.research.google.com/github/ccasazza22/SEO-Workbooks/blob/master/Compare_Pages_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

In [ ]:
pip install tiktoken

In [ ]:
pip install newspaper3k

In [ ]:
pip install transformers

## Establish Functions

Use getContent to pull in the text from the articles you are comparing

In [ ]:
from newspaper import Article
import tiktoken
import openai
import os

openai.api_key = 'INSERT YOUR API KEY'


from transformers import GPT2Tokenizer

# Initialize the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


def getContent(url):
    # Initialize a new Article object
    article = Article(url)

    # Download and parse the article
    article.download()
    article.parse()
    text = article.text
    return text

## Add Articles to Compare Here

In [ ]:
article1 = getContent("YOUR URL") ## add url you are comparing against
article2 = getContent("YOUR COMPETITOR's URL") ## add url you are comparing to
site1 = "YOUR SITE NAME" ## This is the name of your site
site2 = "COMPETITOR SITE NAME" ## be sure to add competitor name


articles_dict = {
    site1: article1,
    site2: article2
  }

## Run Function Here

In [ ]:
truncated_articles = {}
for title, content in articles_dict.items():
    tokens = tokenizer.encode(content, max_length=6000, truncation=True)
    truncated_content = tokenizer.decode(tokens)
    truncated_articles[title] = truncated_content

articles = "\n\n".join([f"{title}: {content}" for title, content in truncated_articles.items()])


def compareURL(articles,site1,site2):
  prompt_start = (
          "You are an AI assistant highlighting the information differences between two articles. You are to help highlight what information is present in one article and not present in the other. Do not make up an answer if there are no significant differences")
  query = "What information does the"+ site2 + "page have that the" + site1 +"page does not?"
  prompt =  [
              {"role": "system", "content": prompt_start},
              {"role": "assistant", "content": articles },
              {"role": "user", "content": query},
          ]
  response = openai.ChatCompletion.create(
          model='gpt-3.5-turbo-16k-0613',
          messages=prompt,
          temperature=0,
          max_tokens=4000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
      )
  answers = response["choices"][0]["message"]["content"]
  return(answers)

print(compareURL(articles,site1,site2))